In [73]:
import itertools

instructions = []
with open('day_22_input') as f:
    for line in f.readlines():
        is_on = line.startswith('on')
        line = line.strip()[line.index('x'):]
        ranges = []
        range_strs = line.split(',')
        for i, range_str in enumerate(range_strs):
            range_str = range_str[2:]
            minstr, _, maxstr = range_str.partition('..')
            ranges.append((int(minstr), int(maxstr)))
        instructions.append((is_on, ranges))
print(instructions)

[(True, [(-48, 6), (-13, 40), (-12, 35)]), (True, [(-44, 0), (-24, 28), (-21, 24)]), (True, [(-26, 28), (-19, 35), (-34, 15)]), (True, [(-33, 14), (-48, 1), (-9, 42)]), (True, [(-15, 34), (-24, 28), (-15, 37)]), (True, [(-5, 40), (-43, 11), (0, 47)]), (True, [(-47, 5), (-19, 34), (-16, 31)]), (True, [(-49, 2), (-42, 7), (-11, 43)]), (True, [(-15, 31), (-6, 48), (-41, 6)]), (True, [(-48, -3), (-18, 36), (-26, 28)]), (False, [(-22, -11), (-42, -27), (-29, -14)]), (True, [(-19, 30), (-45, 0), (-26, 21)]), (False, [(-8, 11), (-4, 15), (-6, 5)]), (True, [(-36, 9), (-34, 16), (-48, 4)]), (False, [(-35, -22), (-35, -17), (-5, 13)]), (True, [(-17, 30), (-11, 35), (-18, 35)]), (False, [(4, 21), (-19, -8), (-29, -18)]), (True, [(-47, 1), (-5, 41), (-13, 40)]), (False, [(18, 31), (-47, -28), (31, 44)]), (True, [(-39, 11), (-2, 49), (-7, 37)]), (True, [(59948, 77225), (-23808, -9387), (7715, 32589)]), (True, [(50928, 60239), (-16409, -5204), (51701, 66341)]), (True, [(-70857, -38119), (-48880, -32

In [74]:
def build_grid():
    grid = []
    for x in range(-50, 51):
        plane = []
        for y in range(-50, 51):
            row = []
            for z in range(-50, 51):
                row.append(False)
            plane.append(row)
        grid.append(plane)
    return grid

In [75]:
minv = -50
maxv = 50

def patch_instruction(ins):
    patched = []
    for rng in ins:
        if rng[0] < minv:
            print('TRIM')
            nmin = minv
        elif rng[0] > maxv:
            print('BOOM')
            return None
        else:
            nmin = rng[0]
        if rng[1] < minv:
            print('BOOM')
            return None
        elif rng[1] > maxv:
            print('TRIM')
            nmax = maxv
        else:
            nmax = rng[1]
        patched.append((nmin, nmax))
    return patched

def score_grid(grid):
    score = 0
    for plane in grid:
        for row in plane:
            for val in row:
                if val:
                    score += 1
    return score
                    
grid = build_grid()
off = abs(minv)
for instruction in instructions:
    is_on, rng = instruction
    patched = patch_instruction(rng)
    if patched is None:
        continue
    # print(patched)
    for x in range(patched[0][0], patched[0][1] + 1):
        for y in range(patched[1][0], patched[1][1] + 1):
            for z in range(patched[2][0], patched[2][1] + 1):
                grid[x + off][y + off][z + off] = is_on
    print(score_grid(grid))


142560
182340
272114
352098
393806
441586
443920
471275
515187
523604
523604
546826
542026
586230
581176
585502
582910
592631
589495
606484
BOOM
BOOM
TRIM
BOOM
BOOM
BOOM
BOOM
TRIM
BOOM
TRIM
TRIM
BOOM
BOOM
TRIM
BOOM
BOOM
BOOM
TRIM
TRIM
BOOM
TRIM
BOOM
BOOM
TRIM
BOOM
TRIM
BOOM
TRIM
BOOM
BOOM
TRIM
BOOM
TRIM
TRIM
TRIM
TRIM
TRIM
BOOM
TRIM
BOOM
TRIM
BOOM
TRIM
BOOM
BOOM
TRIM
BOOM
BOOM
TRIM
TRIM
TRIM
BOOM
TRIM
BOOM
BOOM
TRIM
TRIM
BOOM
TRIM
BOOM
TRIM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
TRIM
TRIM
BOOM
BOOM
TRIM
BOOM
TRIM
TRIM
BOOM
BOOM
BOOM
TRIM
BOOM
TRIM
BOOM
BOOM
TRIM
TRIM
TRIM
BOOM
BOOM
TRIM
BOOM
BOOM
BOOM
TRIM
BOOM
TRIM
BOOM
TRIM
TRIM
BOOM
BOOM
TRIM
BOOM
TRIM
BOOM
BOOM
BOOM
TRIM
BOOM
TRIM
TRIM
TRIM
BOOM
BOOM
TRIM
TRIM
BOOM
BOOM
TRIM
BOOM
BOOM
TRIM
TRIM
TRIM
BOOM
TRIM
BOOM
TRIM
TRIM
TRIM
BOOM
TRIM
BOOM
BOOM
TRIM
BOOM
BOOM
BOOM
TRIM
BOOM
TRIM
BOOM
TRIM
BOOM
TRIM
TRIM
BOOM
TRIM
BOOM
BOOM
BOOM
TRIM
BOOM
TRIM
BOOM
BOOM
BOOM
BOOM
BOOM
TRIM
TRIM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
TRIM
TRIM
BOOM
TRIM
BOOM
TRIM


In [76]:
ndims = 3

class Point(object):
    
    def __init__(self, vals):
        self.vals = vals
        
    def __repr__(self):
        return ','.join([str(val) for val in self.vals])

class Square(object):
    
    def __init__(self, ranges):
        self.start = Point([r[0] for r in ranges])
        self.end = Point([r[1] for r in ranges])
                
    def __repr__(self):
        return f'{{{self.start}->{self.end}}}'
    
    def key(self):
        return tuple(self.start.vals + self.end.vals)
    
# Returns true if pot is contained in exists
def is_in(pot, exists):
    for dim in range(ndims):
        if pot.start.vals[dim] < exists.start.vals[dim] or pot.end.vals[dim] > exists.end.vals[dim]:
            return False
    return True

def disjoint(s1, s2):
    for dim in range(ndims):
        if s1.end.vals[dim] < s2.start.vals[dim]:
            return True
        if s2.end.vals[dim] < s1.start.vals[dim]:
            return True
    return False

def dedupe(squares):
    added_set = set()
    result = []
    for square in squares:
        key = square.key()
        if key not in added_set:
            result.append(square)
            added_set.add(key)
    return result

def intersect(s1, s2):
    sortvals = []
    for dim in range(ndims):
        sortvals.append(sorted((s1.start.vals[dim], s1.end.vals[dim] + 1, s2.start.vals[dim], s2.end.vals[dim] + 1)))
    squares = []
    
    bits = [(0, 1), (1, 2), (2, 3)]
    possibilities = itertools.product(bits, repeat=3)
    for possibility in possibilities:
        subranges = []
        for dim in range(ndims):
            sr_start = sortvals[dim][possibility[dim][0]]
            sr_end = sortvals[dim][possibility[dim][1]]
            if sr_start == sr_end:
                continue
            subrange = (sr_start, sr_end - 1)
            subranges.append(subrange)
        if len(subranges) != ndims:
            continue
        new_square = Square(subranges)
        squares.append(new_square)
    return dedupe(squares)
    
def volume(s):
    mults = []
    for dim in range(ndims):
        mults.append(abs(s.end.vals[dim] - s.start.vals[dim]) + 1)
    result = 1
    for mult in mults:
        result *= mult
    return result
         
def run():
    count_on = 0
    for i, instruction in enumerate(instructions):
        # print(f'Instruction {instruction}')
        is_on, rang = instruction
        if is_on:
            new_ranges = [Square(rang)]
            already_existed = []
            for prev in instructions[:i]:
                # print(f'  Factoring {prev}')
                prev_is_on, prev_range = prev
                prev_square = Square(prev_range)
                if prev_is_on:
                    new_new_ranges = []
                    for sq in new_ranges:
                        # print(f'  Considering {sq}')
                        if disjoint(sq, prev_square):
                            # unrelated, keep
                            new_new_ranges.append(sq)
                            # print(f'  Keeping whole range {sq}')
                            continue
                        if is_in(sq, prev_square):
                            # Fully enveloped, drop
                            already_existed.append(sq)
                            # print(f'  Dropping wholly enveloped range {sq}')
                            continue
                        inter_squares = intersect(sq, prev_square)
                        exist_squares = [s for s in inter_squares if is_in(s, prev_square) and is_in(s, sq)]
                        for exist_square in exist_squares:
                            # print(f'  Dropping overlapped partial {exist_square}')
                            already_existed.append(exist_square)
                        inter_squares = [s for s in inter_squares if not is_in(s, prev_square) and is_in(s, sq)]
                        for inter_square in inter_squares:
                            # print(f'  Keeping private partial {inter_square}')
                            new_new_ranges.append(inter_square)
                    new_ranges = new_new_ranges
                else:
                    new_existing_ranges = []
                    for sq in already_existed:
                        # print(f'  Considering {sq}')
                        if disjoint(sq, prev_square):
                            # unrelated, keep existing
                            # print(f'  Continuing without whole range {sq}')
                            new_existing_ranges.append(sq)
                            continue
                        if is_in(sq, prev_square):
                            # Fully envoloped, existing moves to new
                            new_ranges.append(sq)
                            continue
                        inter_squares = intersect(sq, prev_square)
                        new_squares = [s for s in inter_squares if is_in(s, prev_square) and is_in(s, sq)]
                        for new_square in new_squares:
                            new_ranges.append(new_square)
                        inter_squares = [s for s in inter_squares if not is_in(s, prev_square) and is_in(s, sq)]
                        for inter_square in inter_squares:
                            new_existing_ranges.append(inter_square)
                    already_existed = new_existing_ranges          
            total_new = 0
            for i, a in enumerate(new_ranges):
                for b in new_ranges[i+1:]:
                    if not disjoint(a, b):
                        print(f'Non disjoint range: {a} {b}')
            for new_range in new_ranges:
                # print(f' Adding range {new_range}')
                total_new += volume(new_range)
            count_on += total_new
            print(f'The instruction {instruction} added {total_new} new squares {count_on}')
        else:
            to_delete = []
            already_removed = [Square(rang)]
            for prev in instructions[:i]:
                prev_is_on, prev_range = prev
                prev_square = Square(prev_range)
                if prev_is_on:
                    new_already_removed = []
                    for sq in already_removed:
                        if disjoint(sq, prev_square):
                            # unrelated, keep
                            new_already_removed.append(sq)
                            continue
                        if is_in(sq, prev_square):
                            # Fully enveloped, drop
                            to_delete.append(sq)
                            continue
                        inter_squares = intersect(sq, prev_square)
                        newadd_squares = [s for s in inter_squares if is_in(s, prev_square) and is_in(s, sq)]
                        for newadd_square in newadd_squares:
                            to_delete.append(newadd_square)
                        inter_squares = [s for s in inter_squares if not is_in(s, prev_square) and is_in(s, sq)]
                        for inter_square in inter_squares:
                            new_already_removed.append(inter_square)
                    already_removed = new_already_removed
                else:
                    new_to_delete = []
                    for sq in to_delete:
                        if disjoint(sq, prev_square):
                            # unrelated, keep existing
                            new_to_delete.append(sq)
                            continue
                        if is_in(sq, prev_square):
                            # Fully envoloped, existing moves to new
                            already_removed.append(sq)
                            continue
                        inter_squares = intersect(sq, prev_square)
                        prevdel_squares = [s for s in inter_squares if is_in(s, prev_square) and is_in(s, sq)]
                        for prevdel_square in prevdel_squares:
                            already_removed.append(prevdel_square)
                        inter_squares = [s for s in inter_squares if not is_in(s, prev_square) and is_in(s, sq)]
                        for inter_square in inter_squares:
                            new_to_delete.append(inter_square)
                    to_delete = new_to_delete
            total_delete = 0
            for delete_range in to_delete:
                total_delete += volume(delete_range)
            count_on -= total_delete
            print(f'The instruction {instruction} removed {total_delete} new squares {count_on}')
    print(count_on)

run()
# s1 = Square([(0, 2), (3, 4), (0, 0)])
# s2 = Square([(0, 5), (1, 8), (0, 0)])
# sqs = intersect(s1, s2)
# for sq in sqs:
#     print(f'{sq} {is_in(sq, s1)} {is_in(sq, s2)}')


The instruction (True, [(-48, 6), (-13, 40), (-12, 35)]) added 142560 new squares 142560
The instruction (True, [(-44, 0), (-24, 28), (-21, 24)]) added 39780 new squares 182340
The instruction (True, [(-26, 28), (-19, 35), (-34, 15)]) added 89774 new squares 272114
The instruction (True, [(-33, 14), (-48, 1), (-9, 42)]) added 79984 new squares 352098
The instruction (True, [(-15, 34), (-24, 28), (-15, 37)]) added 41708 new squares 393806
The instruction (True, [(-5, 40), (-43, 11), (0, 47)]) added 47780 new squares 441586
The instruction (True, [(-47, 5), (-19, 34), (-16, 31)]) added 2334 new squares 443920
The instruction (True, [(-49, 2), (-42, 7), (-11, 43)]) added 27355 new squares 471275
The instruction (True, [(-15, 31), (-6, 48), (-41, 6)]) added 43912 new squares 515187
The instruction (True, [(-48, -3), (-18, 36), (-26, 28)]) added 8417 new squares 523604
The instruction (False, [(-22, -11), (-42, -27), (-29, -14)]) removed 0 new squares 523604
The instruction (True, [(-19, 30

The instruction (False, [(53454, 82767), (-21250, -1548), (22267, 54071)]) removed 11083838162383 new squares 1203963142194193
The instruction (False, [(-79117, -60440), (-63622, -38775), (-8955, 15394)]) removed 2441574973439 new squares 1201521567220754
The instruction (True, [(-19536, 6402), (-69997, -47256), (-62149, -43050)]) added 2961957541118 new squares 1204483524761872
The instruction (False, [(19297, 31924), (-76788, -55257), (-63942, -44359)]) removed 448008481833 new squares 1204035516280039
The instruction (True, [(-5012, 24323), (63273, 68750), (-46613, -33442)]) added 646278208686 new squares 1204681794488725
The instruction (True, [(35328, 58056), (31710, 65651), (-34982, -15237)]) added 11032323398271 new squares 1215714117886996
The instruction (True, [(-48192, -33863), (-44827, -29117), (-63183, -34387)]) added 1540721554713 new squares 1217254839441709
The instruction (True, [(-61029, -34553), (-74065, -54148), (9801, 31057)]) added 8293476560507 new squares 122554

The instruction (False, [(8909, 41254), (-93808, -74009), (-4819, 23133)]) removed 13315511000864 new squares 1201055325401803
The instruction (True, [(-52200, -25835), (15960, 30377), (42069, 69104)]) added 3268130971302 new squares 1204323456373105
The instruction (True, [(36591, 45810), (-77352, -60366), (-26787, 1341)]) added 3450422348543 new squares 1207773878721648
The instruction (True, [(-77722, -44666), (14773, 30746), (-61160, -34050)]) added 6370471658437 new squares 1214144350380085
The instruction (False, [(-46582, -35737), (46260, 81259), (-46889, -11312)]) removed 6298344342818 new squares 1207846006037267
The instruction (False, [(-71643, -56716), (14635, 38747), (-61795, -43527)]) removed 5061257171339 new squares 1202784748865928
The instruction (False, [(39240, 74444), (-51811, -27332), (28772, 58190)]) removed 5647183796017 new squares 1197137565069911
The instruction (True, [(-58340, -31522), (-66224, -44479), (23007, 48233)]) added 2949122062646 new squares 12000

The instruction (False, [(-32989, -9724), (37526, 63857), (-70387, -56746)]) removed 2308039361681 new squares 1154497224598763
The instruction (False, [(29903, 51146), (-63108, -50491), (32857, 41295)]) removed 2129392996155 new squares 1152367831602608
The instruction (True, [(-46331, -35553), (-3044, 13283), (62129, 77317)]) added 2493486283768 new squares 1154861317886376
The instruction (True, [(33499, 52700), (-70674, -45077), (-39327, -19649)]) added 8246027569440 new squares 1163107345455816
The instruction (True, [(58723, 70030), (15231, 36339), (-60353, -33130)]) added 992804770836 new squares 1164100150226652
The instruction (True, [(-989, 14391), (-78770, -65176), (22254, 51047)]) added 4889233307688 new squares 1168989383534340
The instruction (False, [(66499, 89004), (7540, 23824), (-22765, -8879)]) removed 3639205833964 new squares 1165350177700376
The instruction (False, [(44823, 66466), (-10101, 16210), (53856, 61509)]) removed 2778267335524 new squares 116257191036485